In [52]:
import pandas as pd
import numpy as np
import pymysql
import json

from janome.tokenizer import Tokenizer
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
    

In [53]:
def connect_sql():

    connection = pymysql.connect(
        host = 'localhost' ,
        user = 'root' ,
        password = 'snskdhkskK123!' ,
        database = 'project_DB'
    )

    query = 'SELECT * FROM ORIG_MRDN_TB'
    df = pd.read_sql(query , connection)

    connection.close()

    return df

In [ ]:
def calc_cosine(prompt):
    prompt = [prompt]
    data = connect_sql()
    tokenizer = Tokenizer()
    tokenized_data = [[token.surface for token in tokenizer.tokenize(sentence)] for sentence in prompt]
    w2v_model = Word2Vec(sentences = tokenized_data , vector_size = 100 , window = 5 , min_count = 1 ,
                         workers = 4)
    
    sentence_embeddings = [
        sum([w2v_model.wv[word] for word in sentence if word in w2v_model.wv]) / len(sentence)
        for sentence in tokenized_data
    ]

    similarity_scores = []
    for idx in range(data.shape[0]):
        similarity_score = cosine_similarity(sentence_embeddings , [json.loads(data.loc[idx , 'EMBEDDED_INFORMATION'])])
        similarity_scores.append((idx , similarity_score))
    return similarity_scores

In [59]:
similarity_data = calc_cosine('肩こり')

/var/folders/cg/w5448dgs6vgg_kqyxn1mmxd80000gn/T/ipykernel_36087/2289350848.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query , connection)
